In [1]:
import numpy as np

def csnedge(gx, gy, boxsize=0.1):
    """
    The normalized statistic of edge x-y.
    
    Parameters:
    gx, gy: Gene expression values of gene x and gene y. If there are n cells, gx and gy are 1D arrays of length n.
    boxsize: Size of neighborhood, Default = 0.1
    
    Returns:
    edge: 1D array of length n, the normalized statistic of edge x-y in all cells.
    """
    
    n = len(gx)
    upper = np.zeros(n)
    lower = np.zeros(n)
    a = np.zeros((2, n))
    B = [None, None]
    
    for i in range(2):
        g = gx if i == 0 else gy
        s1 = np.sort(g)
        s2 = np.argsort(g)
        n0 = n - np.sum(np.sign(s1))
        h = round(boxsize / 2 * np.sum(np.sign(s1)))
        k = 1
        
        while k <= n:
            s = 0
            while k + s + 1 <= n and s1[k + s] == s1[k - 1]:
                s += 1
            if s >= h:
                upper[s2[k - 1:k + s]] = g[s2[k - 1]]
                lower[s2[k - 1:k + s]] = g[s2[k - 1]]
            else:
                upper[s2[k - 1:k + s]] = g[s2[min(n - 1, k + s + h - 1)]]
                lower[s2[k - 1:k + s]] = g[s2[max(n0 * (n0 > h), k - h) - 1]]
            k = k + s + 1
        
        B[i] = (g[:, None] <= upper) & (g[:, None] >= lower)
        a[i, :] = np.sum(B[i], axis=0)
    
    edge = (np.sum(B[0] & B[1], axis=0) * n - a[0, :] * a[1, :]) / np.sqrt(a[0, :] * a[1, :] * (n - a[0, :]) * (n - a[1, :]) / (n - 1))
    
    return edge

In [3]:
import numpy as np
from scipy.stats import norm
from scipy.sparse import csr_matrix

def csnet(data, c=None, alpha=0.01, boxsize=0.1, weighted=0):
    """
    Construction of cell-specific networks.
    
    Parameters:
    data: Gene expression matrix, rows = genes, columns = cells.
    c: Construct the CSNs for all cells if None; Construct the CSN for cell k if c is k.
    alpha: Significant level (e.g., 0.001, 0.01, 0.05 ...); larger alpha leads to more edges, Default = 0.01.
    boxsize: Size of neighborhood, Default = 0.1.
    weighted: 1 if edge is weighted, 0 if edge is not weighted (Default).
    
    Returns:
    csn: Cell-specific network, the kth CSN is in csn[k]; rows = genes, columns = genes.
    """
    
    n1, n2 = data.shape
    if c is None:
        c = range(n2)
    
    # Define the neighborhood of each plot
    upper = np.zeros((n1, n2))
    lower = np.zeros((n1, n2))
    for i in range(n1):
        s1 = np.sort(data[i, :])
        s2 = np.argsort(data[i, :])
        n3 = n2 - np.sum(np.sign(s1))
        h = round(boxsize / 2 * np.sum(np.sign(s1)))
        k = 1
        while k <= n2:
            s = 0
            while k + s + 1 <= n2 and s1[k + s] == s1[k - 1]:
                s += 1
            if s >= h:
                upper[i, s2[k - 1:k + s]] = data[i, s2[k - 1]]
                lower[i, s2[k - 1:k + s]] = data[i, s2[k - 1]]
            else:
                upper[i, s2[k - 1:k + s]] = data[i, s2[min(n2 - 1, k + s + h - 1)]]
                lower[i, s2[k - 1:k + s]] = data[i, s2[max(n3 * (n3 > h), k - h) - 1]]
            k = k + s + 1
    
    # Construction of cell-specific network
    csn = [None] * n2
    B = np.zeros((n1, n2))
    p = -norm.ppf(alpha)
    for k in c:
        for j in range(n2):
            B[:, j] = (data[:, j] <= upper[:, k]) & (data[:, j] >= lower[:, k])
        a = np.sum(B, axis=1)
        d = (B @ B.T * n2 - np.outer(a, a)) / np.sqrt(np.outer(a, a) * np.outer(n2 - a, n2 - a) / (n2 - 1) + np.finfo(float).eps)
        np.fill_diagonal(d, 0)
        if weighted:
            csn[k] = d * (d > 0)
        else:
            csn[k] = csr_matrix(d > p)
        print(f'Cell {k} is completed')
    
    return csn